In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [62]:
df = pd.read_excel('/content/Jobstreet_Data_EDA.xlsx')
df.columns

Index(['index', 'Title', 'Company', 'Location', 'Salary', 'Uploaded Time',
       'Job Level', 'Qualification', 'Experience Years Needed', 'Job Type',
       'Company Specialization', 'Company Scale', 'Processed Time', 'Benefit',
       'Description', 'Link', 'Date Uploaded', 'Time', 'Data Job Type',
       'Description Language', 'Province', 'Python', 'R', 'SQL', 'Tableau',
       'Power_BI', 'Python Only', 'SQL Only', 'Tableau Only', 'Power BI Only',
       'Python SQL Only', 'Python Tableau Only', 'Python Power BI Only',
       'SQL Tableau Only', 'SQL Power BI Only', 'Python SQL Tableau',
       'Python SQL Power BI', 'Day of Week', '24 Hours'],
      dtype='object')

In [63]:
dataset = df[['Salary','Job Level','Qualification','Experience Years Needed','Company Specialization',
              'Company Scale','Processed Time','Province','Python','SQL','Tableau','Power_BI']]
dataset = dataset[~dataset['Salary'].str.contains('-1')]

In [64]:
dataset['Salary'] = dataset['Salary'].apply(lambda x: x.replace('IDR ','').replace(' per bulan','').replace(' jt',''))
dataset['Salary']

2         8 - 10
6        10 - 14
7      4,8 - 5,5
11       5,5 - 8
15         5 - 7
         ...    
474      6,3 - 8
476        5 - 7
488       6 - 12
494    4,5 - 6,3
499        7 - 9
Name: Salary, Length: 130, dtype: object

In [65]:
dataset = dataset[~dataset['Salary'].str.contains('SGD')]

In [66]:
dataset['Min Salary Range'] = dataset['Salary'].apply(lambda x: x.replace(' ','').split('-')[0])
dataset['Max Salary Range'] = dataset['Salary'].apply(lambda x: x.replace(' ','').split('-')[1])

In [67]:
dataset['Min Salary Range'] = dataset['Min Salary Range'].apply(lambda x: x.replace(',','.'))
dataset['Max Salary Range'] = dataset['Max Salary Range'].apply(lambda x: x.replace(',','.'))

In [68]:
dataset['Min Salary Range'] = dataset['Min Salary Range'].astype(float)
dataset['Max Salary Range'] = dataset['Max Salary Range'].astype(float)

In [69]:
dataset['Avg Salary'] = (dataset['Min Salary Range']+dataset['Max Salary Range'])/2

In [70]:
data_final = dataset[['Avg Salary','Processed Time','Job Level','Qualification','Experience Years Needed','Company Specialization',
              'Company Scale','Province','Python','SQL','Tableau','Power_BI']]
data_final = data_final.reset_index()
data_final = data_final.drop(columns=['index'])

In [71]:
data_final = data_final[~data_final['Company Specialization'].str.contains('None')]
data_final = data_final[~data_final['Processed Time'].str.contains('None')]

In [72]:
data_final

,Avg Salary,Processed Time,Job Level,Qualification,Experience Years Needed,Company Specialization,Company Scale,Province,Python,SQL,Tableau,Power_BI
1,12.00,25 hari,Pegawai (non-manajemen & non-supervisor),"Sertifikat Professional, D3 (Diploma), D4 (Dip...",3 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",201 - 500 pekerja,Jakarta Raya,0,0,0,0
6,6.00,29 hari,Pegawai (non-manajemen & non-supervisor),Sarjana (S1),1 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",501 - 1000 pekerja,Jakarta Raya,0,1,0,0
7,5.75,10 hari,Pegawai (non-manajemen & non-supervisor),"Sertifikat Professional, D3 (Diploma), D4 (Dip...",2 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",1- 50 pekerja,Jakarta Raya,0,0,0,0
8,11.00,1 hari,Pegawai (non-manajemen & non-supervisor),"Diploma Pascasarjana, Gelar Professional, Magi...",2 tahun,"Sains, Sains & Teknologi",51 - 200 pekerja,Jakarta Raya,0,0,0,0
9,10.00,16 hari,Pegawai (non-manajemen & non-supervisor),"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",3 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",1- 50 pekerja,Jakarta Raya,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
124,7.15,29 hari,Pegawai (non-manajemen & non-supervisor),"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",2 tahun,"Penjualan / Pemasaran, Pemasaran/Pengembangan ...",2001 - 5000 pekerja,Jakarta Raya,0,0,0,0
125,6.00,22 hari,Pegawai (non-manajemen & non-supervisor),"Sertifikat Professional, D3 (Diploma), D4 (Dip...",2 tahun,"Penjualan / Pemasaran, Penjualan - Teknik/Tekn...",1- 50 pekerja,Jakarta Raya,0,0,0,0
126,9.00,20 hari,Pegawai (non-manajemen & non-supervisor),Sarjana (S1),1 tahun,"Sains, Sains & Teknologi",1- 50 pekerja,Jakarta Raya,1,0,0,0
127,5.40,17 hari,Pegawai (non-manajemen & non-supervisor),Sarjana (S1),2 tahun,"Lainnya, Jurnalis/Editor",201 - 500 pekerja,Jakarta Raya,0,0,0,0


In [73]:
data_final['Processed Time'] = data_final['Processed Time'].apply(lambda x: x.replace(' hari','')).astype(int)
data_final['Processed Time'].value_counts()

29    13
20     9
27     7
22     7
10     7
28     6
1      5
25     5
16     5
13     3
6      3
9      3
21     3
19     3
17     3
7      2
30     2
14     2
15     2
24     2
23     1
26     1
4      1
2      1
18     1
Name: Processed Time, dtype: int64

In [74]:
# Convert day in processed time into week

week = []
first = np.arange(1,8)
second = np.arange(8,15)
third = np.arange(15,22)
fourth = np.arange(22,32)
first
for time in data_final['Processed Time']:
  if time in first:
    week.append(1)
  elif time in second:
    week.append(2)
  elif time in third:
    week.append(3)
  elif time in fourth:
    week.append(4)

In [75]:
data_final['Weeks Processed'] = week

In [76]:
# Now drop the Processed Time column

data_final = data_final.drop(columns=['Processed Time'])

In [77]:
data_final

,Avg Salary,Job Level,Qualification,Experience Years Needed,Company Specialization,Company Scale,Province,Python,SQL,Tableau,Power_BI,Weeks Processed
1,12.00,Pegawai (non-manajemen & non-supervisor),"Sertifikat Professional, D3 (Diploma), D4 (Dip...",3 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",201 - 500 pekerja,Jakarta Raya,0,0,0,0,4
6,6.00,Pegawai (non-manajemen & non-supervisor),Sarjana (S1),1 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",501 - 1000 pekerja,Jakarta Raya,0,1,0,0,4
7,5.75,Pegawai (non-manajemen & non-supervisor),"Sertifikat Professional, D3 (Diploma), D4 (Dip...",2 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",1- 50 pekerja,Jakarta Raya,0,0,0,0,2
8,11.00,Pegawai (non-manajemen & non-supervisor),"Diploma Pascasarjana, Gelar Professional, Magi...",2 tahun,"Sains, Sains & Teknologi",51 - 200 pekerja,Jakarta Raya,0,0,0,0,1
9,10.00,Pegawai (non-manajemen & non-supervisor),"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",3 tahun,"Komputer/Teknologi Informasi, IT-Perangkat Lunak",1- 50 pekerja,Jakarta Raya,0,1,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
124,7.15,Pegawai (non-manajemen & non-supervisor),"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",2 tahun,"Penjualan / Pemasaran, Pemasaran/Pengembangan ...",2001 - 5000 pekerja,Jakarta Raya,0,0,0,0,4
125,6.00,Pegawai (non-manajemen & non-supervisor),"Sertifikat Professional, D3 (Diploma), D4 (Dip...",2 tahun,"Penjualan / Pemasaran, Penjualan - Teknik/Tekn...",1- 50 pekerja,Jakarta Raya,0,0,0,0,4
126,9.00,Pegawai (non-manajemen & non-supervisor),Sarjana (S1),1 tahun,"Sains, Sains & Teknologi",1- 50 pekerja,Jakarta Raya,1,0,0,0,3
127,5.40,Pegawai (non-manajemen & non-supervisor),Sarjana (S1),2 tahun,"Lainnya, Jurnalis/Editor",201 - 500 pekerja,Jakarta Raya,0,0,0,0,3


In [78]:
data_final.columns

Index(['Avg Salary', 'Job Level', 'Qualification', 'Experience Years Needed',
       'Company Specialization', 'Company Scale', 'Province', 'Python', 'SQL',
       'Tableau', 'Power_BI', 'Weeks Processed'],
      dtype='object')

In [79]:
data_final = data_final[~data_final['Company Scale'].str.contains('None')]

In [85]:
# Done, now we're ready for prediction

data_final_dum = pd.get_dummies(data_final)

from sklearn.model_selection import train_test_split

X = data_final_dum.drop('Avg Salary',axis=1)
y = data_final_dum['Avg Salary'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

type(y)

numpy.ndarray

In [88]:
# Random Forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf = RandomForestRegressor()

np.mean(cross_val_score(rf, X_train,y_train,scoring='neg_mean_absolute_error',cv=3))

-1.8400961538461533

In [100]:
# Tuning

from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':np.arange(1,301,10), 'criterion':('mse','mae'), 'max_features':('auto', 'sqrt', 'log2'), 'min_samples_split':np.arange(2,11,1)}
gs = GridSearchCV(rf, parameters, scoring='neg_mean_absolute_error',cv=3)
gs.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, rand...
             param_grid={'criterion': ('mse', 'mae'),
                         'max_features': ('auto', 'sqrt', 'log2'),


In [101]:
gs.best_score_

-1.6740231990231988

In [102]:
gs.best_estimator_

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=21, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [106]:
pred = gs.best_estimator_.predict(X_test)
np.mean(pred)

7.04563492063492